In [2]:
import numpy as np
import numpy as np
import scipy.io as sio

batch_size = 1

def f(X, y, W, lamb = 1e-2):
    F_x = np.dot(X,W)

    C = np.max(F_x, axis=1, keepdims=True)
    exp_scores = np.exp(F_x - C)

    S = exp_scores/np.sum(exp_scores, axis=1, keepdims=True)

    #  log-loss
    log_S = -np.log(S[np.arange(batch_size), y])
    L = np.sum(log_S)/batch_size

    # Add regularization using the L2 norm
    L_reg = 0.5*lamb*np.sum(W*W)
    L+= L_reg

    # Gradient of the loss with respect to scores
    grad = S.copy()
    # Substract 1 from the scores of the correct class
    grad[np.arange(batch_size),y] -= 1
    grad /= batch_size

    # Gradient of the loss with respect to weights
    grad_W = X.T.dot(grad)

    # Add gradient regularization
    grad_W+= lamb*W

    return L, grad_W

In [7]:
import numpy as np
from numpy import linalg as ln
import scipy as sp
import scipy.optimize

def bfgs_method(f, w0, X, y, tot_iter=None, epsi=10e-3):

#     if tot_iter is None:
    tot_iter = 5
    # initial values
    k = 0
    _, gfk = f(X, y, w0)
    N = 7840  ###  Flatten size for MNIST
    m = 1
    I = np.eye(N, dtype=int)
    Hk = np.eye(N, dtype=int)
    wk = w0 
    while ln.norm(gfk) > epsi and k < tot_iter:
        pk = -np.dot(Hk, gfk.flatten()[:,np.newaxis])
        alpha_k = 0.5 #Can add backtrack here
        pk = pk.reshape(784,10)
        wkp1 = wk + alpha_k * pk
        sk = wkp1 - wk
        wk = wkp1
        _, gfkp1 = f(X, y, wkp1)
        yk = gfkp1 - gfk
        gfk = gfkp1
        yk = yk.flatten()
        rho = 1.0 / (yk.dot(sk.flatten()))
        sk = sk.flatten()
        M_1 = I - rho*sk[:, np.newaxis]*yk[np.newaxis, :]
        M_2 = I - rho*yk[:, np.newaxis]*sk[np.newaxis, :]
        Hk = np.dot(M_1, np.dot(Hk, M_2)) + (rho*sk[:, np.newaxis]*sk[np.newaxis, :])    ###  4.25 Pg 131 Lecture notes
        k += 1

    return (wk, k)

X = np.random.randint(10,size = (1,784))
y = 2
W = np.zeros(shape=[784, 10])
result, k = bfgs_method(f,W, X, y)
print('Result of BFGS method:')
print('Final Result (best point): %s' % (result))
print('Iteration Count: %s' % (k))

(7840, 1)
(7840,)
(7840, 1)
(7840,)
(7840, 1)
(7840,)
(7840, 1)
(7840,)
(7840, 1)
(7840,)
Result of BFGS method:
Final Result (best point): [[-0.04364117 -0.04364117  0.39277052 ... -0.04364117 -0.04364117
  -0.04364117]
 [-0.01246891 -0.01246891  0.11222015 ... -0.01246891 -0.01246891
  -0.01246891]
 [-0.03740672 -0.03740672  0.33666045 ... -0.03740672 -0.03740672
  -0.03740672]
 ...
 [-0.03740672 -0.03740672  0.33666045 ... -0.03740672 -0.03740672
  -0.03740672]
 [-0.05611007 -0.05611007  0.50499067 ... -0.05611007 -0.05611007
  -0.05611007]
 [-0.03740672 -0.03740672  0.33666045 ... -0.03740672 -0.03740672
  -0.03740672]]
Iteration Count: 5


[[-138.0806592  -138.0806592  1242.72593284 -138.0806592  -138.0806592
  -138.0806592  -138.0806592  -138.0806592  -138.0806592  -138.0806592 ]]
